# Titanic: Machine Learning from Disaster

In [125]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

np.random.seed(42)

## Importing data

In [126]:
dtypes = {
    "PassengerId" : "int64",
    "Name": "object",
    "Survived": "int64",
    "Pclass" : "object",
    "Ticket" : "object",
    "Sex" : "object",
    "Age" : "float64",
    "SibSp" : "int64",
    "Parch" : "int64",
    "Fare" : "float64",
    "Cabin" : "object",
    "Embarked" : "object"
}

In [160]:
train_df = pd.read_csv('train.csv',sep=',',header=0,usecols=["PassengerId","Name","Survived",
            "Pclass","Ticket","Sex","Age","SibSp","Parch","Fare","Cabin","Embarked"],dtype=dtypes,na_values=[' '])
test_df = pd.read_csv('test.csv',sep=',',header=0,usecols=["PassengerId","Name","Pclass","Sex","Age",
            "Ticket","SibSp","Parch","Fare","Cabin","Embarked"],dtype=dtypes,na_values=[' '])

df = pd.concat([train_df,test_df])
df.set_index("PassengerId",inplace=True)
df.head(3)

,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived,Ticket
PassengerId,,,,,,,,,,,
1,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,3,male,1,0.0,A/5 21171
2,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,female,1,1.0,PC 17599
3,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,female,0,1.0,STON/O2. 3101282


In [161]:
df.dropna(subset=['Embarked'],inplace=True)
df.Age.fillna(df.Age.mean(),inplace=True)
df.Cabin.fillna("Without",inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1307 entries, 1 to 1309
Data columns (total 11 columns):
Age         1307 non-null float64
Cabin       1307 non-null object
Embarked    1307 non-null object
Fare        1306 non-null float64
Name        1307 non-null object
Parch       1307 non-null int64
Pclass      1307 non-null object
Sex         1307 non-null object
SibSp       1307 non-null int64
Survived    889 non-null float64
Ticket      1307 non-null object
dtypes: float64(3), int64(2), object(6)
memory usage: 122.5+ KB


## Some Feature Engineering

Adding some new features that may lead to some more information:
- Fill null ages with information from person's title
- Check if it was a kid (Kids may have more chance of surviving, as they have priority on evacuation)
- Calculate the family size (Bigger families may have trouble on evacuating)
- Get the title from the person's name (Diferentiate tripulants)
- Get what I think it was the deck of the ship cabins (People from lower decks may have less chance of surviving)


In [162]:
df['Alone'] = [0 if x.SibSp+x.Parch > 0 else 1 for index,x in df.iterrows()]
df['MFam'] = [1 if x.SibSp+x.Parch <= 4 and x.SibSp+x.Parch >= 2 else 0 for index,x in df.iterrows()]
df['LFam'] = [1 if x.SibSp+x.Parch > 4 else 0 for index,x in df.iterrows()]
df['Deck'] = [(x.Cabin)[0] for index,x in df.iterrows()]
df['Kid'] = [1 if x.Age < 10 else 0 for index,x in df.iterrows()]
df['Title'] = [x.Name.split(',')[1].split(' ')[1] if (x.Name.split(',')[1].split(' ')[1])[-1] == '.' else 'Common' for index,x in df.iterrows()]
df.head(3)

,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived,Ticket,Alone,MFam,LFam,Deck,Kid,Title
PassengerId,,,,,,,,,,,,,,,,,
1,22.0,Without,S,7.2500,"Braund, Mr. Owen Harris",0,3,male,1,0.0,A/5 21171,0,0,0,W,0,Mr.
2,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,female,1,1.0,PC 17599,0,0,0,C,0,Mrs.
3,26.0,Without,S,7.9250,"Heikkinen, Miss. Laina",0,3,female,0,1.0,STON/O2. 3101282,1,0,0,W,0,Miss.


In [166]:
num_feat = df.select_dtypes('number').columns.values
cat_feat = df.select_dtypes('object').columns.values 
df_num = df[num_feat].drop('Survived',axis=1)
df_cat = df[cat_feat].drop(['Cabin','Ticket','Name'],axis=1)

df_cat = pd.get_dummies(df_cat)

new_df = pd.concat([df_num,df_cat],axis=1)

new_df = (new_df - new_df.min())/(new_df.max()-new_df.min())

final = pd.concat([new_df,df.Survived],axis=1)

final.head(3)

,Age,Fare,Parch,SibSp,Alone,MFam,LFam,Kid,Embarked_C,Embarked_Q,...,Title_Master.,Title_Miss.,Title_Mlle.,Title_Mme.,Title_Mr.,Title_Mrs.,Title_Ms.,Title_Rev.,Title_Sir.,Survived
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0.273456,0.014151,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.473882,0.139136,0.0,0.125,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0.323563,0.015469,0.0,0.000,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [167]:
# Check correlations
corr = final.corr()
corr.style.background_gradient()

It seems that with only these features, the classifier performs the best (Which does not surprises me at all, since classifying women as survivors and men as not, gives 0.7+ accuracy, and predicting that everyone dies gives ~0.67). 

In [ ]:
# Select the features I'm going to use
final = final[['Sex','Survived','Embarked','Pclass']]

train = final[final.Survived.notna()]
test = final[final.Survived.isna()].drop('Survived',axis=1)

train.head(3)

In [88]:
X = train.drop('Survived',axis=1)
y = train.Survived

## Train Classifiers

In [90]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier,RandomForestClassifier
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

In [91]:
param_grid = [
  {'learning_rate': np.linspace(0.1,0.01,5), 'loss': ['deviance','exponential'],
  'n_estimators': np.arange(50,100,5) }
 ]

search = GridSearchCV(GradientBoostingClassifier(),param_grid,cv=5,scoring='accuracy',n_jobs=4).fit(X_train,y_train)
mean = search.cv_results_['mean_test_score'].mean()
print(search.best_score_)
print(mean)
print(search.best_estimator_.score(X_test,y_test))

0.8243243243243243
0.8240315315315317
0.797752808988764


In [92]:
param_grid = [
  {'learning_rate': np.linspace(0.1,0.01,5), 'algorithm' : ['SAMME', 'SAMME.R'],
  'n_estimators': np.arange(50,100,5) }
 ]

search = GridSearchCV(AdaBoostClassifier(),param_grid,cv=5,scoring='accuracy',n_jobs=4).fit(X_train,y_train)
mean = search.cv_results_['mean_test_score'].mean()
print(search.best_score_)
print(mean)
print(search.best_estimator_.score(X_test,y_test))

0.7882882882882883
0.7873423423423422
0.7842696629213484


In [100]:
param_grid = [
  {'n_estimators': np.arange(50,100,10) }
 ]

search = GridSearchCV(RandomForestClassifier(),param_grid,cv=5,scoring='accuracy',n_jobs=4).fit(X_train,y_train)
mean = search.cv_results_['mean_test_score'].mean()
print(search.best_score_)
print(mean)
print(search.best_estimator_.score(X_test,y_test))

0.8243243243243243
0.8243243243243243
0.797752808988764


In [94]:
param_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

search = GridSearchCV(SVC(),param_grid,cv=5,scoring='accuracy',n_jobs=4).fit(X_train,y_train)
mean = search.cv_results_['mean_test_score'].mean()
print(search.best_score_)
print(mean)
print(search.best_estimator_.score(X_test,y_test))

0.8198198198198198
0.7719594594594595
0.797752808988764


## Select the best classifier

It seems that RandomForest peformed the best. Train the final classifier with all data.

In [95]:
param_grid = [
  {'n_estimators': np.arange(10,100,10) }
]

model = GridSearchCV(RandomForestClassifier(),param_grid,cv=10,scoring='accuracy',n_jobs=4).fit(X,y)
mean = model.cv_results_['mean_test_score'].mean()
print(model.best_score_)
print(mean)

0.8198198198198198
0.8110236220472441


## Write Results

In [98]:
#test.Fare.fillna(test.Fare.mean(),inplace=True)

y_predict = model.predict(test)
result_df = pd.DataFrame(columns=['PassengerId','Survived'],
                         dtype=np.int64)
result_df['PassengerId'] = test.index
result_df['Survived'] = y_predict.astype(np.int64)
result_df.to_csv('result.csv',index=False)
result_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0
